In [55]:
import pandas as pd
import numpy as np

## Chargement des fichier de conso

In [56]:
br_in = pd.read_excel(r"C:\Users\sgasmi\Desktop\Dev\Contrôle de gestion sociale\CONSO BRIDGE IN _ CMO.xlsx", sheet_name="base IN CORRIGEE")
br_out = pd.read_excel(r"C:\Users\sgasmi\Desktop\Dev\Contrôle de gestion sociale\CONSO BRIDGE OUT _ CMO.xlsx", sheet_name="base OUT corrigée")

## Préparation des colonnes

In [57]:
# Choix des colonnes à fusionner
Conso_in = br_in[[ "Local payroll  Id","MOIS", "Category", "Contract end Date", "Country", "Legal entity", "Establishment", "Site",
                 "Last name", "First name", "Contract type", "Contract status", "Reason for entry", "Contract start Date"]].copy()


# Ajout des colonnes spécifiques de Conso_Out
Conso_in[["reason for leaving"]] = ""
Conso_in["Mouvement"] = "In" 

# Choix des colonnes à fusionner
Conso_out = br_out[[ "Local payroll  Id", "MOIS", "Cat.", "Contract start Date", "Contract end Date", "Country", "Company", "Establishment", "Site", "Last name", "First name", "Contract type", "Contract status", "reason for leaving"]].copy()
Conso_out.columns=  ["Local payroll  Id", "MOIS", "Category",  "Contract start Date","Contract end Date", "Country", "Legal entity", "Establishment", "Site", "Last name", "First name", "Contract type", "Contract status", "reason for leaving"] 

# Ajout des colonnes spécifiques de Conso_in
Conso_out[["Reason for entry"]] = ""

#Legal entity
Conso_out["Mouvement"] = "Out"

# Remise en ordre des colonnes
l_in = list(Conso_in.columns)
l_in.sort()

Conso_in = Conso_in[l_in]

Conso_out = Conso_out[l_in]

print(f"Le nombre de lignes total est de {len(Conso_in) + len(Conso_out)}. ")

Le nombre de lignes total est de 3248. 


## Fusion des 2 fichiers

In [58]:
Conso = Conso_in.append(Conso_out)
Conso = Conso.reset_index(drop=True)
print(f"Le nombre de lignes total est de {len(Conso)}.")

Le nombre de lignes total est de 3248.


## Controle des sorties/entrées consécutive

In [59]:
Conso = Conso.sort_values(["Local payroll  Id", "MOIS"], ascending=True)
Conso.reset_index(drop=True, inplace=True)
Conso["Anomalies"] = ""
Conso["chg_cat"] = ""

In [60]:
for i, mat in enumerate(Conso.iterrows()):
    
    mat = Conso.at[i, "Local payroll  Id"]
    mvp = Conso.at[i, "Mouvement"]
    
    if i != len(Conso)-1:
        
        if Conso.at[i+1, "Local payroll  Id"] == mat:
            Conso.at[i+1, "chg_cat"] = Conso.at[i, "Category"] + " => " + Conso.at[i, "Category"]
            
            if mvp == Conso.at[i+1, "Mouvement"]:
                Conso.at[i+1, "Anomalies"] = "Incohérence mouvement"

In [61]:
Conso_in = Conso.loc[Conso["Mouvement"]=="In"]
Conso_out = Conso.loc[Conso["Mouvement"]=="Out"]

In [64]:
col_in =["MOIS", "Contract start Date", "Local payroll  Id",  "Reason for entry", "Country", "Legal entity", "Establishment",  "Contract end Date", "Last name", "First name", "Site", "Category","Contract type", "Contract status"]
col_out= [ "MOIS", "Contract start Date", "Local payroll  Id", "reason for leaving", "Contract end Date",  "Category", "Country", "Establishment", "Site", "Legal entity",  "Last name", "First name", "Contract type", "Contract status"]
col_conso = [ "MOIS", "Contract start Date", "Local payroll  Id", "Mouvement",   "Reason for entry",  "reason for leaving", "Contract end Date",  "Category", "Country", "Establishment", "Site", "Legal entity",  "Last name", "First name", "Contract type", "Contract status", "Anomalies"]

Conso_in = Conso_in[col_in] 
Conso_out = Conso_out[col_out]
Conso = Conso[conso]

# Formatage des dates
for col in ["MOIS", "Contract start Date","Contract end Date"] :
    Conso_in[col] = pd.to_datetime(Conso_in[col]).dt.date
    Conso_out[col] = pd.to_datetime(Conso_out[col]).dt.date
    Conso[col] = pd.to_datetime(Conso[col]).dt.date


MOIS                   object
Contract start Date    object
Local payroll  Id      object
Reason for entry       object
Country                object
Legal entity           object
Establishment          object
Contract end Date      object
Last name              object
First name             object
Site                   object
Category               object
Contract type          object
Contract status        object
dtype: object


In [72]:
#Conso.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\Test mouvements.xlsx', index=False)



with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Conso Bridges.xlsx', date_format='dd/mm/yyyy') as writer:  
    Conso.to_excel(writer, sheet_name='Conso', index=False)
    Conso_in.to_excel(writer, sheet_name='Bridge_In', index=False)
    Conso_out.to_excel(writer, sheet_name='Bridge_Out', index=False)